In [1]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
import pyarrow as pa
import os
import pyarrow.dataset as ds
import glob

Load OULAD data and view column headers

In [2]:
courses = pd.read_csv('data/OULAD/courses.csv')
assessments = pd.read_csv('data/OULAD/assessments.csv')
VLEdata = pd.read_csv('data/OULAD/VLE.csv')
studentAssessments = pd.read_csv('data/OULAD/studentAssessment.csv')
studentInfo = pd.read_csv('data/OULAD/studentInfo.csv')
studentRegistration = pd.read_csv('data/OULAD/studentRegistration.csv')
studentVLE = pd.read_csv('data/OULAD/studentVLE.csv')

Info on variables can be found here: https://analyse.kmi.open.ac.uk/open-dataset <br>
<br>
courses.csv<br>
File contains the list of all available modules and their presentations. The columns are:<br>
- *code_module* - code name of the module, which serves as the identifier.<br>
- *code_presentation* - code name of the presentation. It consists of the year and "B" for the presentation starting in February and "J" for the presentation starting in October.<br>
- *length* - length of the module-presentation in days.<br>

The structure of B and J presentations may differ and therefore it is good practice to analyse the B and J presentations separately. Nevertheless, for some presentations the corresponding previous B/J presentation do not exist and therefore the J presentation must be used to inform the B presentation or vice versa. In the dataset this is the case of CCC, EEE and GGG modules.
<br>


assessments.csv<br>
This file contains information about assessments in module-presentations. Usually, every presentation has a number of assessments followed by the final exam. CSV contains columns:<br>
- *code_module* - identification code of the module, to which the assessment belongs.<br>
- *code_presentation* - identification code of the presentation, to which the assessment belongs.<br>
- *id_assessment* - identification number of the assessment.<br>
- *assessment_type* - type of assessment. Three types of assessments exist: Tutor Marked Assessment (TMA), Computer Marked Assessment (CMA) and Final Exam (Exam).<br>
- *date* - information about the final submission date of the assessment calculated as the number of days since the start of the module-presentation. The starting date of the presentation has number 0 (zero).<br>
- *weight* - weight of the assessment in %. Typically, Exams are treated separately and have the weight 100%; the sum of all other assessments is 100%.<br>

If the information about the final exam date is missing, it is at the end of the last presentation week.<br>
<br>


vle.csv<br>
The csv file contains information about the available materials in the VLE. Typically, these are html pages, pdf files, etc. Students have access to these materials online and their interactions with the materials are recorded.
The vle.csv file contains the following columns:<br>
- *id_site* - an identification number of the material.<br>
- *code_module* - an identification code for module.<br>
- *code_presentation* - the identification code of presentation.<br>
- *activity_type* - the role associated with the module material.<br>
- *week_from* - the week from which the material is planned to be used.<br>
- *week_to* - week until which the material is planned to be used.<br>
<br>


studentInfo.csv<br>
This file contains demographic information about the students together with their results. File contains the following columns:<br>
- *code_module* - an identification code for a module on which the student is registered.<br>
- *code_presentation* - the identification code of the presentation during which the student is registered on the module.<br>
- *id_student* - a unique identification number for the student.<br>
- *gender* - the student's gender.<br>
- *region* - identifies the geographic region, where the student lived while taking the module-presentation.<br>
- *highest_education* - highest student education level on entry to the module presentation.<br>
- *imd_band* - specifies the Index of Multiple Depravation band of the place where the student lived during the module-presentation.<br>
- *age_band* - band of the student's age.<br>
- *num_of_prev_attempts* - the number times the student has attempted this module.<br>
- *studied_credits* - the total number of credits for the modules the student is currently studying.<br>
- *disability* - indicates whether the student has declared a disability.<br>
- *final_result* - student's final result in the module-presentation.<br>
<br>


studentRegistration.csv<br>
This file contains information about the time when the student registered for the module presentation. For students who unregistered the unregistered date is also recorded. File contains five columns:<br>
- *code_module* - an identification code for a module.<br>
- *code_presentation* - the identification code of the presentation.<br>
- *id_student* - a unique identification number for the student.<br>
- *date_registration* - the date of student's registration on the module presentation, this is the number of days measured relative to the start of the module-presentation (e.g. the negative value -30 means that the student registered to module presentation 30 days before it started).<br>
- *date_unregistration* - the student's unregistered date from the module presentation, this is the number of days measured relative to the start of the module-presentation. Students, who completed the course have this field empty. Students who unregistered have Withdrawal as the value of the final_result column in the studentInfo.csv file.<br>
<br>


studentAssessment.csv<br>
This file contains the results of students' assessments. If the student does not submit the assessment, no result is recorded. The final exam submissions is missing, if the result of the assessments is not stored in the system.
This file contains the following columns:<br>
- *id_assessment* - the identification number of the assessment.<br>
- *id_student* - a unique identification number for the student.<br>
- *date_submitted* - the date of student submission, measured as the number of days since the start of the module presentation.
- *is_banked* - a status flag indicating that the assessment result has been transferred from a previous presentation.<br>
- *score* - the student's score in this assessment. The range is from 0 to 100. The score lower than 40 is interpreted as Fail. The marks are in the range from 0 to 100.<br>
<br>


studentVle.csv<br>
The studentVle.csv file contains information about each student's interactions with the materials in the VLE.
This file contains the following columns:<br>
- *code_module* - an identification code for a module.<br>
- *code_presentation* - the identification code of the module presentation.<br>
- *id_student* - a unique identification number for the student.<br>
- *id_site* - an identification number for the VLE material.<br>
- *date* - the date of student's interaction with the material measured as the number of days since the start of the module-presentation.<br>
- *sum_click* - the number of times a student interacts with the material in that day.<br>

From Kuzilek et al. (2017): <br>
**studentInfo** can be linked to **studentAssessment**, **studentVle** and **studentRegistration** tables using column *id_student*. <br>
**courses** links to the **assessments**, **studentRegistration**, **vle** and **studentInfo** using identifier columns *code_module* and
*code_presentation*. <br>
**assessments** links to **studentAssessment** using *id_assessment*. <br> 
**vle** to **studentVle** using *id_site*.

In [3]:
print(courses.columns.tolist())
print(assessments.columns.tolist())
print(studentAssessments.columns.tolist())
print(VLEdata.columns.tolist())
print(studentInfo.columns.tolist())
print(studentRegistration.columns.tolist())
print(studentVLE.columns.tolist())

['code_module', 'code_presentation', 'module_presentation_length']
['code_module', 'code_presentation', 'id_assessment', 'assessment_type', 'date', 'weight']
['id_assessment', 'id_student', 'date_submitted', 'is_banked', 'score']
['id_site', 'code_module', 'code_presentation', 'activity_type', 'week_from', 'week_to']
['code_module', 'code_presentation', 'id_student', 'gender', 'region', 'highest_education', 'imd_band', 'age_band', 'num_of_prev_attempts', 'studied_credits', 'disability', 'final_result']
['code_module', 'code_presentation', 'id_student', 'date_registration', 'date_unregistration']
['code_module', 'code_presentation', 'id_student', 'id_site', 'date', 'sum_click']


In [4]:
# Both assessments and studentVLE have 'date' column but are not the same variable. 
# Rename 'date' column in assessments to 'due_date'
assessments = assessments.rename(columns={'date': 'due_date'})

In [5]:
print(courses.shape) 
print(assessments.shape)
print(VLEdata.shape)
print(studentAssessments.shape)
print(studentInfo.shape) 
print(studentRegistration.shape) 
print(studentVLE.shape)

(22, 3)
(206, 6)
(6364, 6)
(173912, 5)
(32593, 12)
(32593, 5)
(10655280, 6)


Methods from Casalino et al. 2024: <br>
(1) In detail, the initial step involved merging the student_info and student_vle tables based on
the features code_module , code_presentation , and id_student , resulting in a consolidated table
that integrated information from both sources. <br> 
(2) Subsequently, this consolidated table was merged
with the vle table, utilizing code_module , code_presentation , and id_site as reference points to
create a table containing information on activity types. <br> 
(3) The assessments and student_assessment
tables were merged using the id_assessment feature. <br> 
(4) Finally, the two resulting tables were
joined using the id_student and date features, thereby creating a comprehensive dataset that
encapsulates relevant student interactions and assessment results.

Note that Casalino et al. 2024 do not include studentRegistration information, nor do they include all the assessment information. 

Modified approach <br>
1. Merge studentInfo, studentRegistration, courses into **df_studentInfo**
    - 1 row per ('code_module','code_presentation','id_student')
    - same number of rows as studentInfo & studentRegistration
2. Merge assessments & studentAssessments into **df_assessment**
    - 1 row per ('code_module','code_presentation','id_student','id_assessment')
    - same number of rows as studentAssessments
3. Merge studentVLE & VLEdata into **df_vle**
    - 1 row per ('code_module','code_presentation','id_student','id_site')
    - same number of rows as studentVLE
4. Collapse id_site and pivot df_vle such that:
    - 1 row per ('code_module','code_presentation','id_student','date')
    - activity_type becomes columns with values = sum(sum_click) for that date
5. Merge df_vle & df_assessment into **df_vle_assessment**
    - 1 row per ('code_module','code_presentation','id_student','date','id_assessment')
    - date_submitted from df_assessment is matched to 'date' from df_vle and then dropped
6. Merge df_vle_assessment & df_studentInfo into **df**
    - 1 row per ('code_module','code_presentation','id_student','date','id_assessment')

In [6]:
# Merge studentInfo & studentRegistration dataframes 
common_columns = list(set(studentInfo.columns) & set(studentRegistration.columns)) # ['code_module','code_presentation','id_student']
df_studentInfo=pd.merge(studentInfo, studentRegistration, how='outer', on=common_columns)

# checks on merge
assert studentInfo.shape[1] + studentRegistration.shape[1] - len(common_columns) == df_studentInfo.shape[1]
assert studentInfo.shape[0] == studentRegistration.shape[0] == df_studentInfo.shape[0]

# Merge courses with above (adds column module_presentation_length)
common_columns = list(set(courses.columns) & set(df_studentInfo.columns)) # ['code_module', 'code_presentation']
df_studentInfo=pd.merge(courses, df_studentInfo, how='outer', on=common_columns)

df_studentInfo.head()

,code_module,code_presentation,module_presentation_length,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,date_registration,date_unregistration
0,AAA,2013J,268,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass,-159.0,NaN
1,AAA,2013J,268,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass,-53.0,NaN
2,AAA,2013J,268,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,Withdrawn,-92.0,12.0
3,AAA,2013J,268,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass,-52.0,NaN
4,AAA,2013J,268,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass,-176.0,NaN


In [7]:
# Merge assessments, studentAssessment dataframes 
common_columns = list(set(assessments.columns) & set(studentAssessments.columns)) # ['id_assessment']
df_assessment=pd.merge(studentAssessments, assessments, how='left', on=common_columns)

df_assessment.head()

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,due_date,weight
0,1752,11391,18,0,78.0,AAA,2013J,TMA,19.0,10.0
1,1752,28400,22,0,70.0,AAA,2013J,TMA,19.0,10.0
2,1752,31604,17,0,72.0,AAA,2013J,TMA,19.0,10.0
3,1752,32885,26,0,69.0,AAA,2013J,TMA,19.0,10.0
4,1752,38053,19,0,79.0,AAA,2013J,TMA,19.0,10.0


In [8]:
# Merge studentVLE, VLEdata dataframes 
common_columns = list(set(studentVLE.columns) & set(VLEdata.columns)) # ['id_site', 'code_presentation', 'code_module']
df_vle = pd.merge(studentVLE,VLEdata, how='left', on=common_columns)

df_vle.head()

,code_module,code_presentation,id_student,id_site,date,sum_click,activity_type,week_from,week_to
0,AAA,2013J,28400,546652,-10,4,forumng,NaN,NaN
1,AAA,2013J,28400,546652,-10,1,forumng,NaN,NaN
2,AAA,2013J,28400,546652,-10,1,forumng,NaN,NaN
3,AAA,2013J,28400,546614,-10,11,homepage,NaN,NaN
4,AAA,2013J,28400,546714,-10,1,oucontent,NaN,NaN


Following chunks that are commented out were used to create the dataset version where id_site is still included. 

In [9]:
# Merge df_studentInfo, df_assessment dataframes 
# common_columns = list(set(df_studentInfo.columns) & set(df_assessment.columns)) # ['code_module', 'id_student', 'code_presentation']
# df_student_assessment=pd.merge(df_assessment, df_studentInfo, how='left', on=common_columns)

# df_student_assessment.head()

In [10]:
# # Downcast to smaller data types based on your column stats
# df_vle = df_vle.astype({
#     "id_student": "int32",
#     "id_site": "int32",
#     "date": "int16",
#     "sum_click": "int16",
#     "week_from": "Int8", # nullable integer type (Int8, Int16, Int32) instead of plain numpy ints allows for NaNs
#     "week_to": "Int8", # nullable integer type (Int8, Int16, Int32) instead of plain numpy ints allows for NaNs
#     "code_module": "category", 
#     "code_presentation": "category", 
#     "activity_type": "category"
# })

# # df_vle["code_module"] = df_vle["code_module"].astype("category")
# # df_vle["code_presentation"] = df_vle["code_presentation"].astype("category")
# # df_vle["activity_type"] = df_vle["activity_type"].astype("category")

# df_student_assessment = df_student_assessment.astype({
#     "id_assessment": "int32",
#     "id_student": "int32",
#     "date_submitted": "int16",
#     "is_banked": "int8",
#     "module_presentation_length": "int16",
#     "num_of_prev_attempts": "int8",
#     "studied_credits": "int16",
#     "score": "float32",
#     "due_date": "Int16",              # could use 'Int16' if you want integer + NaNs
#     "weight": "float32",
#     "date_registration": "float32",     # safer with NaNs
#     "date_unregistration": "float32"
# })

# categorical_cols = [
#     "code_module", "code_presentation", "assessment_type",
#     "gender", "region", "highest_education", "imd_band",
#     "age_band", "disability", "final_result"
# ]
# df_student_assessment[categorical_cols] = df_student_assessment[categorical_cols].astype("category")

In [11]:
# # Merge df_student_assessment, df_vle dataframes 
# common_columns = list(set(df_student_assessment.columns) & set(df_vle.columns)) # ['code_module', 'id_student', 'code_presentation']
# df=pd.merge(df_vle, df_student_assessment, how='left', on=common_columns)
# df.head()

# # I ran into a memory issue trying to run this initially
# # MemoryError: Unable to allocate 7.33 GiB for an array with shape (11, 89457269) and data type float64 
# # Added the code chunk above to downcast variables to smaller data types to save memory and now it works

In [ ]:
# df.to_csv("merged_data_ABF.csv",index=False)

Collapsing id_site such that all interactions for each activity_type occurring on the same date are summed. 

In [ ]:
# clean up variables
del assessments, common_columns, courses, studentAssessments, studentInfo, studentRegistration, studentVLE, VLEdata

In [13]:
df_vle = df_vle.astype({
    "id_student": "int32",
    "id_site": "int32",
    "date": "int16",
    "sum_click": "int16",
    "week_from": "Int8", 
    "week_to": "Int8",
    "code_module": "string", 
    "code_presentation": "string", 
    "activity_type": "string"
})

df_vle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10655280 entries, 0 to 10655279
Data columns (total 9 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   code_module        string
 1   code_presentation  string
 2   id_student         int32 
 3   id_site            int32 
 4   date               int16 
 5   sum_click          int16 
 6   activity_type      string
 7   week_from          Int8  
 8   week_to            Int8  
dtypes: Int8(2), int16(2), int32(2), string(3)
memory usage: 406.5 MB


In [ ]:
# Create parquet files for each (code_module, code_presentation) combination
output_path = "data/OULAD/studentVLE_partitioned/"

for module, pres in df_vle[["code_module", "code_presentation"]].drop_duplicates().itertuples(index=False):
    print(module, pres)
    
    df_part = df_vle[(df_vle["code_module"] == module) & (df_vle["code_presentation"] == pres)]

    # print(df_part.head())
    
    folder = os.path.join(output_path, 
                          f"code_module={module}",
                          f"code_presentation={pres}")
    os.makedirs(folder, exist_ok=True)

    # Enforce consistent dtypes
    df_part = df_part.astype({
        "id_student": "int32",
        "date": "int16",
        "code_module": "string",
        "code_presentation": "string"
    })

    # print(df_part.dtypes)   
    
    file_path = os.path.join(folder, "data.parquet")
    df_part.to_parquet(file_path, index=False, engine="pyarrow", use_dictionary=False)

print("✅ Done: partitions written with code_module & code_presentation kept as columns")

In [ ]:
# Check files
dataset = ds.dataset("data/OULAD/studentVLE_partitioned/", format="parquet")
df_vle_part = dataset.to_table().to_pandas()
df_vle_part.info()
df_vle_part.isna().sum()

# Get list of all activity types across the dataset to use later on
all_activities = df_vle["activity_type"].unique().tolist()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10655280 entries, 0 to 10655279
Data columns (total 9 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   code_module        string
 1   code_presentation  string
 2   id_student         int32 
 3   id_site            int32 
 4   date               int16 
 5   sum_click          int16 
 6   activity_type      string
 7   week_from          Int8  
 8   week_to            Int8  
dtypes: Int8(2), int16(2), int32(2), string(3)
memory usage: 406.5 MB


In [ ]:
# Reads in each (code_module, code_presentation) parquet file to perform groupby and pivot operations 
# (which were too memory intensive to do on the entire dataset at once)
# saves output as new parquet file

dataset = ds.dataset("data/OULAD/studentVLE_partitioned/", format="parquet",partitioning="hive")

modules = dataset.partitioning.dictionaries[0].to_pylist()  # code_module values
presentations = dataset.partitioning.dictionaries[1].to_pylist()  # code_presentation values

# Check
# print(modules)
# print(presentations)

input_base = "data/OULAD/studentVLE_partitioned"
output_base = "data/OULAD/studentVLE_agg_partitioned"

# Find all parquet files in the partitioned structure
paths = glob.glob(f"{input_base}/code_module=*/code_presentation=*/data.parquet")
paths = [os.path.normpath(p) for p in paths]

# Check
# print(paths)

for path in paths:
    # Parse partition values from folder names
    parts = path.split(os.sep)
    code_module = parts[-3].split("=")[1]
    code_presentation = parts[-2].split("=")[1]

    # Read file
    df = pd.read_parquet(path)

    # Group and sum
    agged = (
        df.groupby(
            ["id_student", "code_module", "code_presentation", "date", "activity_type"],
            as_index=False, 
            observed=True
        )["sum_click"]
        .sum()
    )
    # Pivot so activity_type becomes columns
    pivoted = agged.pivot_table(
        index=["id_student", "code_module", "code_presentation", "date"],
        columns="activity_type",
        values="sum_click",
        fill_value=0, 
        observed = True
    ).reset_index()

    # Ensure activity_type columns are part of flat column index
    pivoted.columns.name = None

    # Reindex to ensure *all* activity types are present, even if missing
    pivoted = pivoted.reindex(
        columns=["id_student", "code_module", "code_presentation", "date", *all_activities],
        fill_value=0
    )

    # Enforce consistent dtypes
    pivoted = pivoted.astype({
        "id_student": "int32",
        "date": "int16",
        "code_module": "string",
        "code_presentation": "string"
    })

    # Force activity columns to int16
    for col in pivoted.columns:
        if col not in ["id_student", "date", "code_module", "code_presentation"]:
            pivoted[col] = pivoted[col].astype("int16")    

    # Write result to same Hive-style folder structure
    out_dir = os.path.join(
        output_base,
        f"code_module={code_module}",
        f"code_presentation={code_presentation}"
    )
    os.makedirs(out_dir, exist_ok=True)
    out_path = os.path.join(out_dir, "data.parquet")
    pivoted.to_parquet(out_path, engine="pyarrow", index=False)

    print(f"✅ Wrote pivoted {out_path} with {len(pivoted)} rows")

In [ ]:
# Read in all parquet files to recreate full VLE interactions data table that is now grouped and pivoted
dataset = ds.dataset(output_base, format="parquet")
df_vle_agg = dataset.to_table().to_pandas()
print(df_vle_agg.info())
df_vle_agg.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1808119 entries, 0 to 1808118
Data columns (total 24 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   id_student         int32 
 1   code_module        string
 2   code_presentation  string
 3   date               int16 
 4   forumng            int16 
 5   homepage           int16 
 6   oucontent          int16 
 7   subpage            int16 
 8   url                int16 
 9   resource           int16 
 10  glossary           int16 
 11  dataplus           int16 
 12  oucollaborate      int16 
 13  quiz               int16 
 14  ouelluminate       int16 
 15  sharedsubpage      int16 
 16  questionnaire      int16 
 17  page               int16 
 18  externalquiz       int16 
 19  ouwiki             int16 
 20  dualpane           int16 
 21  repeatactivity     int16 
 22  folder             int16 
 23  htmlactivity       int16 
dtypes: int16(21), int32(1), string(2)
memory usage: 106.9 MB
None


,id_student,date,forumng,homepage,oucontent,subpage,url,resource,glossary,dataplus,...,ouelluminate,sharedsubpage,questionnaire,page,externalquiz,ouwiki,dualpane,repeatactivity,folder,htmlactivity
count,1.808119e+06,1.808119e+06,1.808119e+06,1.808119e+06,1.808119e+06,1.808119e+06,1.808119e+06,1.808119e+06,1.808119e+06,1.808119e+06,...,1.808119e+06,1.808119e+06,1.808119e+06,1.808119e+06,1.808119e+06,1.808119e+06,1.808119e+06,1.808119e+06,1.808119e+06,1.808119e+06
mean,7.265757e+05,9.917092e+01,4.409771e+00,3.843256e+00,6.198045e+00,1.886813e+00,3.134207e-01,6.139707e-01,4.864835e-02,2.625270e-02,...,2.158486e-02,9.457342e-05,3.581844e-02,3.519182e-02,3.555739e-02,4.947197e-01,1.145721e-02,4.977548e-06,2.997590e-03,5.109730e-03
std,5.763850e+05,7.507524e+01,9.782323e+00,7.571462e+00,1.557085e+01,5.034384e+00,1.022888e+00,7.196825e+00,9.684412e-01,6.130285e-01,...,4.717104e-01,1.159245e-02,4.484971e-01,3.138031e-01,3.795993e-01,4.194868e+00,2.134152e-01,4.004841e-03,6.323366e-02,1.067792e-01
min,6.516000e+03,-2.500000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.058990e+05,3.200000e+01,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,5.883520e+05,9.300000e+01,0.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,6.458400e+05,1.590000e+02,5.000000e+00,5.000000e+00,6.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,2.698588e+06,2.690000e+02,1.224000e+03,6.977000e+03,3.733000e+03,3.967000e+03,3.600000e+02,4.956000e+03,2.410000e+02,7.000000e+01,...,1.180000e+02,5.000000e+00,2.500000e+01,4.400000e+01,3.320000e+02,5.810000e+02,3.900000e+01,4.000000e+00,7.000000e+00,3.100000e+01


In [ ]:
# Check current data types
# print(df_vle_agg.dtypes)
# print(df_studentInfo.dtypes)
# print(df_assessment.dtypes)

# Downcast to smaller data types 
df_vle_agg = df_vle_agg.astype({
    "code_module": "category", 
    "code_presentation": "category"
})

df_studentInfo = df_studentInfo.astype({
    "id_student": "int32",
    "module_presentation_length": "int32",
    "num_of_prev_attempts": "int32",
    "studied_credits": "int32",
    "date_registration": "float32",     # safer with NaNs
    "date_unregistration": "float32"
})

categorical_cols = [
    "code_module", "code_presentation", 
    "gender", "region", "highest_education", "imd_band",
    "age_band", "disability", "final_result"
]
df_studentInfo[categorical_cols] = df_studentInfo[categorical_cols].astype("category")

df_assessment = df_assessment.astype({
    "id_assessment": "int32",
    "id_student": "int32",
    "date_submitted": "int32",
    "is_banked": "int8",
    "score": "float32",
    "due_date": "float32",              
    "weight": "float32",
    "code_module": "category", 
    "code_presentation": "category",
    "assessment_type": "category"
})

In [21]:
print(df_vle_agg.shape)
print(df_studentInfo.shape)
print(df_assessment.shape)

(1808119, 24)
(32593, 15)
(173912, 10)


In [21]:
# Merge VLE data with assessment data by matching assessment date_submitted with VLE date 
df_vle_assessment = pd.merge(
    df_vle_agg,
    df_assessment,
    left_on=["id_student", "code_module", "code_presentation", "date"],
    right_on=["id_student", "code_module", "code_presentation", "date_submitted"],
    how="outer"  # keeps all df_vle_agg rows
)
# Fill NaNs in VLE activity columns with 0
vle_cols = ["forumng", "homepage", "oucontent", "subpage", "url", "resource",
            "dataplus", "glossary", "oucollaborate", "ouelluminate", 
            "sharedsubpage", "questionnaire", "page", "externalquiz", 
            "ouwiki", "dualpane", "repeatactivity", "folder", "htmlactivity"]

for col in vle_cols:
    if col in df_vle_assessment.columns:
        df_vle_assessment[col] = df_vle_assessment[col].fillna(0).astype("int16")

# Combine date columns into a single one
df_vle_assessment["date"] = df_vle_assessment["date"].combine_first(df_vle_assessment["date_submitted"])
df_vle_assessment = df_vle_assessment.drop(columns=["date_submitted"])

# Optional: reset index
df_vle_assessment.reset_index(drop=True, inplace=True)

df_vle_assessment.shape
# NB: A student that submits multiple assessments on the same day will have two rows with the same date

(1875496, 30)

In [27]:
# Merge df_vle_assessment, df_studentInfo dataframes 
common_columns = list(set(df_vle_assessment.columns) & set(df_studentInfo.columns)) # ['code_module', 'id_student', 'code_presentation']
df=pd.merge(df_vle_assessment, df_studentInfo, how='left', on=common_columns)
print(df.shape)
df.head()

(1875496, 42)


,id_student,code_module,code_presentation,date,forumng,homepage,oucontent,subpage,url,resource,...,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,date_registration,date_unregistration
0,6516,AAA,2014J,-23.0,0,3,23,2,0,0,...,Scotland,HE Qualification,80-90%,55<=,0,60,N,Pass,-52.0,NaN
1,6516,AAA,2014J,-22.0,33,13,34,0,0,2,...,Scotland,HE Qualification,80-90%,55<=,0,60,N,Pass,-52.0,NaN
2,6516,AAA,2014J,-20.0,13,12,8,1,0,7,...,Scotland,HE Qualification,80-90%,55<=,0,60,N,Pass,-52.0,NaN
3,6516,AAA,2014J,-17.0,0,2,0,3,2,0,...,Scotland,HE Qualification,80-90%,55<=,0,60,N,Pass,-52.0,NaN
4,6516,AAA,2014J,-12.0,1,1,0,0,0,0,...,Scotland,HE Qualification,80-90%,55<=,0,60,N,Pass,-52.0,NaN


In [ ]:
df.to_csv("merged_streaming_data.csv", index=False)

In [32]:
# Checks which students submit multiple assessments on the same date
dup_counts = df.groupby(
    ["id_student", "code_module", "code_presentation", "date"]
)["id_assessment"].nunique()

print(dup_counts[dup_counts > 1])

id_student  code_module  code_presentation  date  
8462        DDD          2014J              -1.0      4
24186       GGG          2014B               159.0    2
                                             207.0    3
26211       FFF          2014B               37.0     2
27189       FFF          2014B               138.0    3
                                                     ..
2697921     FFF          2013J               220.0    3
                                             227.0    2
                         2014J               231.0    4
2698125     FFF          2013J               223.0    3
                                             227.0    2
Name: id_assessment, Length: 7615, dtype: int64


In [29]:
# Search for dates where student submits an assignment but has no vle interactions 
df.loc[(df[vle_cols] == 0).all(axis=1) & df["score"].notna()]

,id_student,code_module,code_presentation,date,forumng,homepage,oucontent,subpage,url,resource,...,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,date_registration,date_unregistration
215,8462,DDD,2014J,-1.0,0,0,0,0,0,0,...,London Region,HE Qualification,30-40%,55<=,1,60,N,Withdrawn,-38.0,18.0
216,8462,DDD,2014J,-1.0,0,0,0,0,0,0,...,London Region,HE Qualification,30-40%,55<=,1,60,N,Withdrawn,-38.0,18.0
217,8462,DDD,2014J,-1.0,0,0,0,0,0,0,...,London Region,HE Qualification,30-40%,55<=,1,60,N,Withdrawn,-38.0,18.0
218,8462,DDD,2014J,-1.0,0,0,0,0,0,0,...,London Region,HE Qualification,30-40%,55<=,1,60,N,Withdrawn,-38.0,18.0
246,11391,AAA,2013J,115.0,0,0,0,0,0,0,...,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass,-159.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1875287,2698535,CCC,2014B,69.0,0,0,0,0,0,0,...,Wales,Lower Than A Level,50-60%,0-35,0,60,N,Withdrawn,-156.0,180.0
1875298,2698535,CCC,2014B,139.0,0,0,0,0,0,0,...,Wales,Lower Than A Level,50-60%,0-35,0,60,N,Withdrawn,-156.0,180.0
1875397,2698535,EEE,2013J,171.0,0,0,0,0,0,0,...,Wales,Lower Than A Level,50-60%,0-35,0,60,N,Pass,-74.0,NaN
1875418,2698577,BBB,2014J,51.0,0,0,0,0,0,0,...,Wales,Lower Than A Level,50-60%,35-55,0,60,N,Fail,16.0,NaN
